In [28]:
import numpy as np
from scipy.integrate import quad
from scipy.integrate import cumulative_trapezoid

In [29]:
class simulation2:
    def __init__(self, k_array, E0, A_func, A0, omega, t0, t, t_step):
        self.k_grid = np.array(k_array)
        self.E0 = E0
        self.A = A_func
        self.A0 = A0
        self.omega = omega
        self.t0 = t0
        self.t = t
        self.t_step = t_step
        self.t_array = np.linspace(t0,t,t_step)
    def fourier_phi(self,k):
        return 2**(5/2) / np.sqrt(np.pi) * 1/(np.dot(k, k)+1)**2/np.sqrt(4*np.pi)
    def inner(self, t_prime,k):
        term1 = np.dot(k, k) / 2 * t_prime
        term2 = np.dot(k, self.A0 * np.sin(self.omega * t_prime) * np.array([1.0, 0.0, 0.0])) / self.omega
        term3 = self.A0**2 / 4 * t_prime
        term4 = self.A0**2 / 8 / self.omega * np.sin(2 * self.omega * t_prime)
        return term1 + term2 + term3 + term4
    
    def inner2(self, k):
        def phase(t_pp):
            A_vals = self.A(t_pp)  # shape (N, 3)
            total = A_vals + k
            return np.sum(total * total, axis=1)  # |k + A(t)|^2 for each time

        phase_array = phase(self.t_array)
        
        cum_int =  cumulative_trapezoid(phase_array, self.t_array, initial=0)
        return cum_int

    def outer(self,t_prime,k):
        A_val = self.A(np.array(t_prime))
        def exp_tot (t_prime):
            return np.exp(1j* self.inner(t_prime,k)/2 -1j*self.E0 *(t_prime-self.t0))
        return np.dot(k, A_val) * exp_tot(t_prime)
    
    def time_integral(self,k):
        t_start = self.t0
        t_end = self.t

        def f_tot(t_prime):
            return self.outer(t_prime,k)
        def f_real (t_prime):
            return np.real( f_tot(t_prime))
        def f_imag (t_prime):
            return np.imag(f_tot(t_prime))
        
        # Integrate real and imaginary parts separately
        real_integral, _ = quad(f_real, t_start, t_end)
        imag_integral, _ = quad(f_imag, t_start, t_end)

        # Combine into complex result
        complex_integral = real_integral + 1j * imag_integral

        return -1j * self.fourier_phi(k)*complex_integral
    
    def calc_matrix(self):
        results = []
        for k in self.k_grid:
            res = self.time_integral(k)
            results.append(np.abs(res)**2)
        results = np.array(results)
        return results.reshape(len(np.unique(self.k_grid[:,0])),
                           len(np.unique(self.k_grid[:,1])),
                           len(np.unique(self.k_grid[:,2])))

    




In [30]:
I = 1e-2/3.51 #au
omega_L = 0.057 #au
A0 = np.sqrt(I)/omega_L
E0 = -0.5 #au
eps = 0

def A(t):
    t = np.asarray(t)
    print(t)
    sin_t = np.sin(omega_L * t)
    A_vals = np.zeros((len(t), 3))  # shape (N, 3)

    A_vals[:, 0] = 0.0
    A_vals[:, 1] = sin_t * np.sin(omega_L * t) * eps
    A_vals[:, 2] = sin_t * np.cos(omega_L * t)

    return A0 / np.sqrt(1 + eps**2) * A_vals

def A2(t):
    return A0/np.sqrt(1+eps**2) * np.sin(omega_L * t) * np.array([ 0.0,np.sin(omega_L * t)*eps, np.cos(omega_L * t)])


k_x = np.linspace(0, 1, 30)  # 11 Werte von -5 bis 5
k_y = np.linspace(0, 1, 3)
k_z = np.linspace(0, 5, 60)

# Erzeuge das Gitter
X,Y,Z = np.meshgrid(k_x, k_y,k_z)
k_grid = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=-1)


In [34]:
A(np.array([8]))

[8]


array([[0.       , 0.       , 0.3702283]])

In [36]:
A2(8)

array([0.       , 0.       , 0.3702283])

In [31]:

sim2 = simulation2(k_grid,E0,A,A0,omega_L,0,2,10)
#sim2.calc_matrix()
#sim2.inner(np.array([1,0,0]))
#sim2.t_array

In [32]:
import matplotlib.pyplot as plt

# Assuming `matrix` is your result from `calc_matrix()`
matrix = sim2.calc_matrix()

# Slice at k_z = 0 (only one slice in this case)
plt.imshow(matrix[:, 0, :], extent=(k_x[0], k_x[-1], k_z[0], k_z[-1]), origin='lower', cmap='viridis')
plt.colorbar(label='|Amplitude|²')
plt.xlabel('$k_x$')
plt.ylabel('$k_z$')
plt.title('Momentum Amplitude Squared')
plt.show()


1.0


TypeError: len() of unsized object

In [ ]:
plt.plot(matrix[0, 0, :])

NameError: name 'matrix' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Assuming `matrix` is your result from `calc_matrix()`
matrix = sim2.calc_matrix()

# Slice at k_z = 0 (only one slice in this case)
plt.imshow(matrix[:, :, 0], extent=(k_x[0], k_x[-1], k_y[0], k_y[-1]), origin='lower', cmap='viridis')
plt.colorbar(label='|Amplitude|²')
plt.xlabel('$k_x$')
plt.ylabel('$k_y$')
plt.title('Momentum Amplitude Squared')
plt.show()


TypeError: len() of unsized object